# imports

In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta

import psycopg2

from scipy.stats import kruskal, mannwhitneyu, spearmanr

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

pd.options.display.float_format = "{:.6f}".format
np.random.seed(42)

: 

In [ ]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [ ]:
ve = """
    SELECT 
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date AS dt_emissao,
    SUM(pp.qtde_ven) AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V'
    and pp.dt_emissao < DATE '2026-01-01'
    AND pp.cd_loja NOT IN ('01','08','09')
    and c.codcli not in ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011','99999','88888','21097')
    AND pp.dt_emissao >= DATE '2019-01-01'
    and pp.codvde not in ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
GROUP BY
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date;
"""

In [ ]:
dv = """
   SELECT 
    pe.cd_produto AS cod_pro,
    pe.cd_loja,
    e.dt_emissao::date AS dt_emissao,
    SUM(pe.qt_devolve) AS devolucoes
FROM "D-1".prod_ent pe
JOIN "D-1".entrada e
    ON e.cd_loja = pe.cd_loja
    AND e.sg_serie = pe.sg_serie
    AND e.nu_nota = pe.nu_nota
JOIN "D-1".cliente cli
    ON cli.codcli = pe.cd_cliente
WHERE
    e.dt_emissao >= DATE '2019-01-01'
    and e.dt_emissao < DATE '2026-01-01'
    AND e.in_cancela = 'N'
    AND e.in_clifor = 'C'
    AND UPPER(e.nfeenvstat) NOT LIKE '%DENEG%'
    AND pe.cd_cfop NOT IN ('1949', '2949', '1603')
    AND cli.codcli NOT IN ('99999','88888','21097')
    AND cli.codcid <> '0501'
    AND cli.codarea <> '112'
    AND pe.cd_loja NOT IN ('01','08','09')
GROUP BY
    pe.cd_produto,
    pe.cd_loja,
    e.dt_emissao::date;
"""

In [ ]:
query3 = """
select pp.cd_produto AS cod_pro, pro.dt_virada, pro.dt_valida 
from"D-1".prod_pro pp
join "D-1".promocao pro on pp.sq_promoca = pro.sq_promoca and pp.cd_tploja = pro.cd_tploja
where pro.cd_tploja = '01'
"""

In [ ]:
with psycopg2.connect(**conn_params) as conn:
    promo = pd.read_sql(query3, conn)

In [ ]:
with psycopg2.connect(**conn_params) as conn:
    ven = pd.read_sql(ve, conn)

In [ ]:
with psycopg2.connect(**conn_params) as conn:
    dev = pd.read_sql(dv, conn)

In [ ]:
dev

In [ ]:
gaso = pd.read_excel("gasolina.xlsx")

In [ ]:
gaso

In [ ]:
datas = pd.read_excel("calendario.xlsx")
datas

# tratamento

In [13]:
ven['dt_emissao'] = pd.to_datetime(ven['dt_emissao'])
dev['dt_emissao'] = pd.to_datetime(dev['dt_emissao'])

ven['vendas'] = ven['vendas'].astype(float)
dev['devolucoes'] = dev['devolucoes'].astype(float)

In [14]:
for df in [ven, dev]:
    df['cod_pro'] = df['cod_pro'].astype(str).str.zfill(6)  # se seu cod tem padding
    df['cd_loja'] = df['cd_loja'].astype(str).str.zfill(2)

In [15]:
ven_agg = (
    ven
    .groupby(['cod_pro', 'cd_loja', 'dt_emissao'], as_index=False)
    .agg({'vendas': 'sum'})
)

dev_agg = (
    dev
    .groupby(['cod_pro', 'cd_loja', 'dt_emissao'], as_index=False)
    .agg({'devolucoes': 'sum'})
)

In [16]:
base = ven_agg.merge(
    dev_agg,
    on=['cod_pro', 'cd_loja', 'dt_emissao'],
    how='left'
)

base['devolucoes'] = base['devolucoes'].fillna(0)
base['vendas_liquidas'] = base['vendas'] - base['devolucoes']

In [17]:
base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas
0,000028,03,2021-06-07,1.000000,0.000000,1.000000
1,000028,03,2022-08-26,2.000000,0.000000,2.000000
2,000028,03,2022-09-16,1.000000,0.000000,1.000000
3,000028,03,2023-04-06,1.000000,0.000000,1.000000
4,000028,03,2023-10-26,1.000000,0.000000,1.000000
...,...,...,...,...,...,...
5241008,111551,05,2025-12-26,1.000000,0.000000,1.000000
5241009,111551,06,2025-12-26,1.000000,0.000000,1.000000
5241010,111552,04,2025-12-23,1.000000,0.000000,1.000000
5241011,111552,05,2025-12-23,3.000000,0.000000,3.000000


In [18]:
gaso['MÊS'] = pd.to_datetime(gaso['MÊS'])
inicio = pd.to_datetime('2019-01-01')
fim = pd.to_datetime('2025-12-31')

gaso_gc = (
    gaso[(gaso['PRODUTO'] == 'GASOLINA COMUM') &
         (gaso['MÊS'] >= inicio) &
         (gaso['MÊS'] <= fim)]
    [['MÊS', 'PREÇO MÉDIO REVENDA']]
    .rename(columns={'PREÇO MÉDIO REVENDA': 'preco_gasolina'})
    .copy()
)


In [19]:
gaso_gc

,MÊS,preco_gasolina
1,2019-01-01,4.159000
5,2019-02-01,4.075000
9,2019-03-01,4.251000
13,2019-04-01,4.374000
17,2019-05-01,4.473000
...,...,...
367,2025-07-01,6.440000
372,2025-08-01,6.480000
377,2025-09-01,6.360000
382,2025-10-01,6.310000


In [20]:
gaso_gc["MÊS"] = gaso_gc["MÊS"].dt.to_period("M").dt.to_timestamp()
preco_mes = gaso_gc.set_index("MÊS")["preco_gasolina"].sort_index()

base["MÊS"] = base["dt_emissao"].dt.to_period("M").dt.to_timestamp()
base["preco_gasolina"] = base["MÊS"].map(preco_mes)

In [21]:
if base["preco_gasolina"].isna().any():
    # cria uma série mensal completa e ffill
    idx = pd.date_range(base["MÊS"].min(), base["MÊS"].max(), freq="MS")
    preco_mes_full = preco_mes.reindex(idx).ffill()
    base["preco_gasolina"] = base["MÊS"].map(preco_mes_full)


In [22]:
datas['DATA'] = pd.to_datetime(datas['DATA'])

In [23]:
base = base.merge(
    datas[['DATA', 'TIPO_DIA', 'DIA_SEMANA']],
    left_on='dt_emissao',
    right_on='DATA',
    how='left'
).drop(columns=['DATA'])

In [24]:
promo["cod_pro"] = promo["cod_pro"].astype(str).str.zfill(6)
promo["dt_virada"] = pd.to_datetime(promo["dt_virada"])
promo["dt_valida"] = pd.to_datetime(promo["dt_valida"])

promo = promo.drop_duplicates(subset=["cod_pro", "dt_virada", "dt_valida"]).copy()

In [25]:
base_promo = base.merge(promo, on="cod_pro", how="left")
base_promo["em_promocao"] = (
    (base_promo["dt_emissao"] >= base_promo["dt_virada"]) &
    (base_promo["dt_emissao"] <= base_promo["dt_valida"])
).astype(int)

In [26]:
base = (
    base_promo
    .groupby(['cod_pro','cd_loja','dt_emissao'], as_index=False)
    .agg({
        'vendas': 'first',
        'devolucoes': 'first',
        'vendas_liquidas': 'first',
        'MÊS': 'first',
        'preco_gasolina': 'first',
        'TIPO_DIA': 'first',
        'DIA_SEMANA': 'first',
        'em_promocao': 'max'
    })
)

base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas,MÊS,preco_gasolina,TIPO_DIA,DIA_SEMANA,em_promocao
0,000028,03,2021-06-07,1.000000,0.000000,1.000000,2021-06-01,5.751000,DIA COMUM,Segunda-feira,0
1,000028,03,2022-08-26,2.000000,0.000000,2.000000,2022-08-01,5.340000,DIA COMUM,Sexta-feira,0
2,000028,03,2022-09-16,1.000000,0.000000,1.000000,2022-09-01,4.890000,DIA COMUM,Sexta-feira,0
3,000028,03,2023-04-06,1.000000,0.000000,1.000000,2023-04-01,5.580000,PRE FERIADO,Quinta-feira,0
4,000028,03,2023-10-26,1.000000,0.000000,1.000000,2023-10-01,5.610000,DIA COMUM,Quinta-feira,0
...,...,...,...,...,...,...,...,...,...,...,...
5241008,111551,05,2025-12-26,1.000000,0.000000,1.000000,2025-12-01,6.350000,POS FERIADO,Sexta-feira,0
5241009,111551,06,2025-12-26,1.000000,0.000000,1.000000,2025-12-01,6.350000,POS FERIADO,Sexta-feira,0
5241010,111552,04,2025-12-23,1.000000,0.000000,1.000000,2025-12-01,6.350000,FERIAS,Terça-feira,0
5241011,111552,05,2025-12-23,3.000000,0.000000,3.000000,2025-12-01,6.350000,FERIAS,Terça-feira,0


In [27]:
base['ano'] = base['dt_emissao'].dt.year

vendas_anuais = (
    base
    .groupby(['cod_pro', 'ano'])['vendas_liquidas']
    .sum()
    .reset_index()
)

In [28]:
media_anual_produto = (
    vendas_anuais
    .groupby('cod_pro')['vendas_liquidas']
    .mean()
    .reset_index(name='media_anual')
)

produtos_validos = media_anual_produto.loc[
    media_anual_produto['media_anual'] > 108,
    'cod_pro'
]

base = base[base['cod_pro'].isin(produtos_validos)].copy()

In [29]:
base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas,MÊS,preco_gasolina,TIPO_DIA,DIA_SEMANA,em_promocao,ano
317,000045,03,2019-12-06,10.000000,0.000000,10.000000,2019-12-01,4.513000,DIA COMUM,Sexta-feira,0,2019
318,000045,03,2019-12-13,30.000000,0.000000,30.000000,2019-12-01,4.513000,DIA COMUM,Sexta-feira,0,2019
319,000045,03,2020-01-07,10.000000,0.000000,10.000000,2020-01-01,4.500000,FERIAS,Terça-feira,0,2020
320,000045,03,2020-01-27,10.000000,0.000000,10.000000,2020-01-01,4.500000,FERIAS,Segunda-feira,0,2020
321,000045,03,2020-01-28,30.000000,0.000000,30.000000,2020-01-01,4.500000,FERIAS,Terça-feira,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
5240960,111239,07,2025-12-10,24.000000,0.000000,24.000000,2025-12-01,6.350000,DIA COMUM,Quarta-feira,0,2025
5240961,111239,07,2025-12-12,24.000000,24.000000,0.000000,2025-12-01,6.350000,DIA COMUM,Sexta-feira,1,2025
5240962,111239,07,2025-12-13,6.000000,0.000000,6.000000,2025-12-01,6.350000,DIA COMUM,Sábado,1,2025
5240963,111239,07,2025-12-19,2.000000,0.000000,2.000000,2025-12-01,6.350000,FERIAS,Sexta-feira,0,2025


In [30]:
resultado_tipo_dia = []
for loja, df_loja in base.groupby("cd_loja"):
    grupos = [
        g["vendas_liquidas"].values
        for _, g in df_loja.groupby("TIPO_DIA")
        if len(g) > 5
    ]
    if len(grupos) >= 2:
        _, p = kruskal(*grupos)
        resultado_tipo_dia.append({"cd_loja": loja, "teste": "TIPO_DIA", "p_valor": p})
resultado_tipo_dia = pd.DataFrame(resultado_tipo_dia)

In [31]:
resultado_dia_semana = []
for loja, df_loja in base.groupby("cd_loja"):
    grupos = [
        g["vendas_liquidas"].values
        for _, g in df_loja.groupby("DIA_SEMANA")
        if len(g) > 5
    ]
    if len(grupos) >= 2:
        _, p = kruskal(*grupos)
        resultado_dia_semana.append({"cd_loja": loja, "teste": "DIA_SEMANA", "p_valor": p})
resultado_dia_semana = pd.DataFrame(resultado_dia_semana)

In [32]:
resultados_promo = []
for loja, df_loja in base.groupby("cd_loja"):
    g0 = df_loja[df_loja["em_promocao"] == 0]["vendas_liquidas"]
    g1 = df_loja[df_loja["em_promocao"] == 1]["vendas_liquidas"]
    if len(g0) > 30 and len(g1) > 30:
        _, p = mannwhitneyu(g0, g1)
        resultados_promo.append({
            "cd_loja": loja,
            "teste": "PROMOCAO",
            "p_valor": p,
            "media_sem_promo": g0.mean(),
            "media_com_promo": g1.mean(),
            "uplift_abs": g1.mean() - g0.mean(),
            "uplift_pct": (g1.mean() / g0.mean() - 1) * 100
        })
resultados_promo = pd.DataFrame(resultados_promo)


In [33]:
resultado_preco = []
for loja, df_loja in base.groupby("cd_loja"):
    df_loja = df_loja.dropna(subset=["preco_gasolina", "vendas_liquidas"])
    if len(df_loja) > 10:
        corr, p = spearmanr(df_loja["preco_gasolina"], df_loja["vendas_liquidas"])
        resultado_preco.append({"cd_loja": loja, "correlacao_spearman": corr, "p_valor": p})
resultado_preco = pd.DataFrame(resultado_preco)

In [34]:
resumo_testes = (
    resultado_tipo_dia
    .merge(resultado_dia_semana, on="cd_loja", how="outer", suffixes=("_tipo_dia", "_dia_semana"))
    .merge(resultado_preco, on="cd_loja", how="outer")
    .merge(resultados_promo, on="cd_loja", how="outer")
)

print("\n===== RESUMO TESTES (amostra) =====")
print(resumo_testes.head(10))



===== RESUMO TESTES (amostra) =====
  cd_loja teste_tipo_dia  p_valor_tipo_dia teste_dia_semana  \
0      03       TIPO_DIA          0.000126       DIA_SEMANA   
1      04       TIPO_DIA          0.000001       DIA_SEMANA   
2      05       TIPO_DIA          0.000034       DIA_SEMANA   
3      06       TIPO_DIA          0.000000       DIA_SEMANA   
4      07       TIPO_DIA          0.001432       DIA_SEMANA   

   p_valor_dia_semana  correlacao_spearman  p_valor_x     teste  p_valor_y  \
0            0.000000             0.014452   0.000000  PROMOCAO   0.000000   
1            0.000000             0.002337   0.025626  PROMOCAO   0.000000   
2            0.000000             0.003583   0.001399  PROMOCAO   0.000000   
3            0.000000            -0.073454   0.000000  PROMOCAO   0.000000   
4            0.000000            -0.025722   0.000000  PROMOCAO   0.000000   

   media_sem_promo  media_com_promo  uplift_abs  uplift_pct  
0         3.636382        11.792358    8.155977  224.

# XGBoost por loja


In [ ]:
corte = pd.to_datetime("2025-01-01")
features_1 = ["preco_gasolina", "em_promocao", "TIPO_DIA", "DIA_SEMANA"]
target = "vendas_liquidas"

In [ ]:
resultados_1step = []
importancias_1step = []

# evita groupby gigante (que estoura RAM). Itera por lista de lojas.
lojas = pd.Series(base["cd_loja"].astype(str).unique()).sort_values().tolist()

for loja in lojas:
    df_loja = base.loc[
        base["cd_loja"].astype(str) == str(loja),
        ["dt_emissao"] + features_1 + [target]
    ].copy()

    if df_loja.empty:
        continue

    df_loja["dt_emissao"] = pd.to_datetime(df_loja["dt_emissao"])
    df_loja = df_loja.sort_values("dt_emissao")

    train = df_loja[df_loja["dt_emissao"] < corte].copy()
    test  = df_loja[df_loja["dt_emissao"] >= corte].copy()

    if len(train) < 500 or len(test) < 50:
        del df_loja, train, test
        gc.collect()
        continue

    # validação temporal (últimos 90 dias do treino)
    val_start = corte - pd.Timedelta(days=90)
    subtrain = train[train["dt_emissao"] < val_start].copy()
    val      = train[train["dt_emissao"] >= val_start].copy()

    use_early_stop = (len(subtrain) >= 300) and (len(val) >= 50)
    if not use_early_stop:
        subtrain = train
        val = None

    # X/y
    X_tr = subtrain[features_1].copy()
    y_tr = subtrain[target].astype(np.float32).values

    X_te = test[features_1].copy()
    y_te = test[target].astype(np.float32).values

    # tipagem (categorical nativo)
    X_tr["TIPO_DIA"] = X_tr["TIPO_DIA"].astype(str).astype("category")
    X_tr["DIA_SEMANA"] = X_tr["DIA_SEMANA"].astype(str).astype("category")
    X_tr["preco_gasolina"] = X_tr["preco_gasolina"].astype(np.float32)
    X_tr["em_promocao"] = X_tr["em_promocao"].astype(np.float32)

    cats_td = X_tr["TIPO_DIA"].cat.categories
    cats_ds = X_tr["DIA_SEMANA"].cat.categories

    # alinha TESTE às categorias do treino (e filtra o que for “categoria nova”)
    X_te["TIPO_DIA"] = pd.Categorical(X_te["TIPO_DIA"].astype(str), categories=cats_td)
    X_te["DIA_SEMANA"] = pd.Categorical(X_te["DIA_SEMANA"].astype(str), categories=cats_ds)
    mask_te = X_te["TIPO_DIA"].notna() & X_te["DIA_SEMANA"].notna()
    X_te = X_te.loc[mask_te].copy()
    y_te = y_te[mask_te.values]

    X_te["preco_gasolina"] = X_te["preco_gasolina"].astype(np.float32)
    X_te["em_promocao"] = X_te["em_promocao"].astype(np.float32)

    if use_early_stop:
        X_val = val[features_1].copy()
        y_val = val[target].astype(np.float32).values

        X_val["TIPO_DIA"] = pd.Categorical(X_val["TIPO_DIA"].astype(str), categories=cats_td)
        X_val["DIA_SEMANA"] = pd.Categorical(X_val["DIA_SEMANA"].astype(str), categories=cats_ds)
        mask_val = X_val["TIPO_DIA"].notna() & X_val["DIA_SEMANA"].notna()
        X_val = X_val.loc[mask_val].copy()
        y_val = y_val[mask_val.values]

        # se a validação ficou pequena, desliga early stopping
        if len(X_val) < 50:
            use_early_stop = False
            X_val = None
            y_val = None
        else:
            X_val["preco_gasolina"] = X_val["preco_gasolina"].astype(np.float32)
            X_val["em_promocao"] = X_val["em_promocao"].astype(np.float32)

    params = dict(
        n_estimators=4000,
        learning_rate=0.03,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=10,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
        eval_metric="mae",
        enable_categorical=True,
        tree_method="hist",
    )
    if use_early_stop:
        params["early_stopping_rounds"] = 200

    model = XGBRegressor(**params)

    if use_early_stop:
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    else:
        model.fit(X_tr, y_tr)

    pred = np.maximum(model.predict(X_te), 0.0)

    mae = mean_absolute_error(y_te, pred)
    rmse = float(np.sqrt(mean_squared_error(y_te, pred)))

    best_it = getattr(model, "best_iteration", None)
    best_it = int(best_it) if best_it is not None else np.nan

    resultados_1step.append({
        "cd_loja": str(loja),
        "mae": float(mae),
        "rmse": rmse,
        "n_train_total": int(len(train)),
        "n_subtrain": int(len(subtrain)),
        "n_val": int(len(val)) if (use_early_stop and val is not None) else 0,
        "n_test": int(len(test)),
        "best_iteration": best_it
    })

    # importâncias (gain) com nomes reais
    score = model.get_booster().get_score(importance_type="gain")
    if len(score) > 0:
        cols = list(X_tr.columns)
        mapa = {f"f{i}": cols[i] for i in range(len(cols))}
        imp = pd.DataFrame({
            "feature": [mapa.get(k, k) for k in score.keys()],
            "importance": list(score.values()),
            "cd_loja": str(loja)
        })
        importancias_1step.append(imp)

    # limpa por loja
    del df_loja, train, test, subtrain, val, X_tr, X_te, model
    if use_early_stop and ("X_val" in locals()) and X_val is not None:
        del X_val
    gc.collect()

df_resultados_1step = pd.DataFrame(resultados_1step).sort_values("rmse").reset_index(drop=True)

if len(importancias_1step):
    df_importancias_1step = pd.concat(importancias_1step, ignore_index=True)
    top10_importancias_1step = (
        df_importancias_1step.groupby("feature", as_index=False)["importance"]
        .mean()
        .sort_values("importance", ascending=False)
        .head(10)
    )
else:
    df_importancias_1step = pd.DataFrame()
    top10_importancias_1step = pd.DataFrame()

print("===== RESULTADOS XGB (1-step) =====")
print(df_resultados_1step.head(20))

===== RESULTADOS XGB (1-step) =====
  cd_loja      mae      rmse  n_train_total  n_subtrain  n_val  n_test  \
0      03 4.087356 11.815088         423605      393119  30486  123402   
1      07 6.355006 16.950938          97122       59092  38030  161870   
2      04 6.248420 20.056424         676545      618303  58242  235166   
3      05 6.914648 21.810055         556985      499360  57625  238142   
4      06 6.929545 23.057829         524998      477820  47178  193499   

   best_iteration  
0               0  
1             708  
2             324  
3               0  
4             366  


In [48]:
print("\n===== TOP 10 IMPORTÂNCIAS (média entre lojas) =====")
print(top10_importancias_1step)


===== TOP 10 IMPORTÂNCIAS (média entre lojas) =====
                    feature    importance
10              em_promocao 132415.103418
4         DIA_SEMANA_Sábado   2619.458438
11           preco_gasolina   2318.518030
0   DIA_SEMANA_Quarta-feira   1813.857727
3    DIA_SEMANA_Sexta-feira   1781.505847
9      TIPO_DIA_PRE FERIADO   1492.594855
8      TIPO_DIA_POS FERIADO   1445.428271
6          TIPO_DIA_FERIADO   1151.946619
1   DIA_SEMANA_Quinta-feira   1148.032172
7           TIPO_DIA_FERIAS   1140.798083


# forecast recursivo

In [ ]:

corte = pd.to_datetime("2025-01-01")
horizonte_max = 30
features_2 = ["cod_pro", "preco_gasolina", "em_promocao", "lag_7", "media_7", "TIPO_DIA", "DIA_SEMANA"]
target = "vendas_liquidas"

# se True, grava previsões (CSV) em vez de segurar na RAM
GUARDAR_PREVISOES = True
OUT_DIR = "outputs_forecast"
os.makedirs(OUT_DIR, exist_ok=True)
CSV_PREV = os.path.join(OUT_DIR, "previsoes_recursivo.csv")

if GUARDAR_PREVISOES:
    with open(CSV_PREV, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["cd_loja","cod_pro","data","venda_prevista","real"])

resultados_forecast = []
amostra_prev = []  # só pra printar umas linhas sem pesar

# evita groupby gigante
lojas = pd.Series(base["cd_loja"].astype(str).unique()).sort_values().tolist()

for loja in lojas:
    df_loja = base.loc[
        base["cd_loja"].astype(str) == str(loja),
        ["cod_pro","dt_emissao","preco_gasolina","em_promocao","TIPO_DIA","DIA_SEMANA",target]
    ].copy()

    if df_loja.empty:
        continue

    df_loja["dt_emissao"] = pd.to_datetime(df_loja["dt_emissao"])
    df_loja["cod_pro"] = df_loja["cod_pro"].astype(str).str.zfill(6)
    df_loja = df_loja.sort_values(["cod_pro","dt_emissao"])

    # split principal
    train_raw = df_loja[df_loja["dt_emissao"] < corte].copy()
    test_raw  = df_loja[df_loja["dt_emissao"] >= corte].copy()

    if len(train_raw) < 5000 or len(test_raw) < 500:
        del df_loja, train_raw, test_raw
        gc.collect()
        continue

    # =========================
    # 1) FEATURES (SÓ NO TREINO) -> SEM VAZAR
    # =========================
    tr = train_raw.sort_values(["cod_pro","dt_emissao"]).copy()

    tr["lag_7"] = tr.groupby("cod_pro")[target].shift(7)
    tr["media_7"] = (
        tr.groupby("cod_pro")[target]
          .apply(lambda s: s.shift(1).rolling(7).mean())
          .reset_index(level=0, drop=True)
    )

    tr_feat = tr.dropna(subset=["lag_7","media_7","preco_gasolina","TIPO_DIA","DIA_SEMANA"]).copy()
    if len(tr_feat) < 2000:
        del df_loja, train_raw, test_raw, tr, tr_feat
        gc.collect()
        continue

    # validação temporal: últimos 90 dias ANTES do corte (ainda dentro do treino)
    val_start = corte - pd.Timedelta(days=90)
    tr_sub  = tr_feat[tr_feat["dt_emissao"] < val_start].copy()
    val_sub = tr_feat[tr_feat["dt_emissao"] >= val_start].copy()

    use_early_stop = (len(tr_sub) >= 2000) and (len(val_sub) >= 300)
    if not use_early_stop:
        tr_sub = tr_feat
        val_sub = None

    # =========================
    # 2) TREINO XGB (categorical nativo)
    # =========================
    X_tr = tr_sub[features_2].copy()
    y_tr = tr_sub[target].astype(np.float32).values

    X_tr["cod_pro"] = X_tr["cod_pro"].astype(str).astype("category")
    X_tr["TIPO_DIA"] = X_tr["TIPO_DIA"].astype(str).astype("category")
    X_tr["DIA_SEMANA"] = X_tr["DIA_SEMANA"].astype(str).astype("category")
    for c in ["preco_gasolina","em_promocao","lag_7","media_7"]:
        X_tr[c] = X_tr[c].astype(np.float32)

    cats_cod = X_tr["cod_pro"].cat.categories
    cats_td  = X_tr["TIPO_DIA"].cat.categories
    cats_ds  = X_tr["DIA_SEMANA"].cat.categories
    seen_cod = set(map(str, cats_cod))

    if use_early_stop:
        # filtra VAL para não ter categoria inédita
        val_sub = val_sub[
            val_sub["cod_pro"].astype(str).isin(seen_cod) &
            val_sub["TIPO_DIA"].astype(str).isin(set(map(str, cats_td))) &
            val_sub["DIA_SEMANA"].astype(str).isin(set(map(str, cats_ds)))
        ].copy()

        if len(val_sub) < 300:
            use_early_stop = False
            val_sub = None
        else:
            X_val = val_sub[features_2].copy()
            y_val = val_sub[target].astype(np.float32).values

            X_val["cod_pro"] = pd.Categorical(X_val["cod_pro"].astype(str), categories=cats_cod)
            X_val["TIPO_DIA"] = pd.Categorical(X_val["TIPO_DIA"].astype(str), categories=cats_td)
            X_val["DIA_SEMANA"] = pd.Categorical(X_val["DIA_SEMANA"].astype(str), categories=cats_ds)
            for c in ["preco_gasolina","em_promocao","lag_7","media_7"]:
                X_val[c] = X_val[c].astype(np.float32)

    params = dict(
        n_estimators=6000,
        learning_rate=0.03,
        max_depth=9,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=10,
        reg_lambda=1.0,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
        eval_metric="mae",
        enable_categorical=True,
        tree_method="hist",
    )
    if use_early_stop:
        params["early_stopping_rounds"] = 250

    model = XGBRegressor(**params)
    if use_early_stop:
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    else:
        model.fit(X_tr, y_tr)
    booster = model.get_booster()

    # =========================
    # 3) HISTÓRICO REAL ATÉ O CORTE (por produto)
    # =========================
    hist_real = (
        train_raw.sort_values(["cod_pro","dt_emissao"])
                .groupby("cod_pro")[target]
                .apply(list)
                .to_dict()
    )

    # filtra teste por produtos vistos no treino do modelo (evita erro e faz sentido pro recursivo)
    te = test_raw[test_raw["cod_pro"].astype(str).isin(seen_cod)].copy()
    if te.empty:
        del df_loja, train_raw, test_raw, tr, tr_feat, tr_sub, model, booster, X_tr
        if use_early_stop and ("X_val" in locals()) and X_val is not None:
            del X_val
        gc.collect()
        continue

    te = te.sort_values(["cod_pro","dt_emissao"])[
        ["cod_pro","dt_emissao","preco_gasolina","em_promocao","TIPO_DIA","DIA_SEMANA",target]
    ].copy()

    # =========================
    # 4) WALK-FORWARD RECURSIVO (métricas streaming)
    # =========================
    n_loja = 0
    sum_abs = 0.0
    sum_sq = 0.0

    if GUARDAR_PREVISOES:
        fprev = open(CSV_PREV, "a", newline="", encoding="utf-8")
        wprev = csv.writer(fprev)

    for cod_pro, df_p in te.groupby("cod_pro", sort=False):
        if cod_pro not in hist_real or len(hist_real[cod_pro]) < 7:
            continue

        df_p = df_p.head(horizonte_max)
        serie = np.asarray(hist_real[cod_pro], dtype=np.float32)

        for row in df_p.itertuples(index=False):
            lag_7 = float(serie[-7])
            media_7 = float(np.mean(serie[-7:]))

            X_fut = pd.DataFrame([{
                "cod_pro": str(cod_pro),
                "preco_gasolina": float(row.preco_gasolina),
                "em_promocao": float(row.em_promocao),
                "lag_7": lag_7,
                "media_7": media_7,
                "TIPO_DIA": str(row.TIPO_DIA),
                "DIA_SEMANA": str(row.DIA_SEMANA),
            }])

            # tipagem idêntica ao treino
            X_fut["cod_pro"] = pd.Categorical(X_fut["cod_pro"], categories=cats_cod)
            X_fut["TIPO_DIA"] = pd.Categorical(X_fut["TIPO_DIA"], categories=cats_td)
            X_fut["DIA_SEMANA"] = pd.Categorical(X_fut["DIA_SEMANA"], categories=cats_ds)
            for c in ["preco_gasolina","em_promocao","lag_7","media_7"]:
                X_fut[c] = X_fut[c].astype(np.float32)

            y_pred = float(booster.inplace_predict(X_fut)[0])
            if y_pred < 0:
                y_pred = 0.0

            real = float(getattr(row, target))
            err = y_pred - real
            sum_abs += abs(err)
            sum_sq += err * err
            n_loja += 1

            # atualiza estado recursivo
            serie = np.append(serie, np.float32(y_pred))

            if GUARDAR_PREVISOES:
                wprev.writerow([str(loja), str(cod_pro), pd.to_datetime(row.dt_emissao).date(), y_pred, real])
            elif len(amostra_prev) < 20:
                amostra_prev.append([str(loja), str(cod_pro), pd.to_datetime(row.dt_emissao).date(), y_pred, real])

    if GUARDAR_PREVISOES:
        fprev.close()

    if n_loja >= 100:
        mae = float(sum_abs / n_loja)
        rmse = float(np.sqrt(sum_sq / n_loja))
        best_it = getattr(model, "best_iteration", None)
        best_it = int(best_it) if best_it is not None else np.nan

        resultados_forecast.append({
            "cd_loja": str(loja),
            "mae": mae,
            "rmse": rmse,
            "n_train": int(len(train_raw)),
            "n_prev": int(n_loja),
            "best_iteration": best_it
        })

    # limpa por loja
    del df_loja, train_raw, test_raw, tr, tr_feat, tr_sub, te, hist_real, X_tr, model, booster
    if use_early_stop and ("X_val" in locals()) and X_val is not None:
        del X_val
    gc.collect()

df_resultados_forecast = pd.DataFrame(resultados_forecast).sort_values("rmse").reset_index(drop=True)

print("\n===== RESULTADOS FORECAST RECURSIVO (por loja) =====")
print(df_resultados_forecast.head(30))

: 

In [ ]:
if GUARDAR_PREVISOES:
    print(f"\nPrevisões gravadas em: {CSV_PREV}")
else:
    df_amostra = pd.DataFrame(amostra_prev, columns=["cd_loja","cod_pro","data","venda_prevista","real"])
    print("\n===== PREVISÕES (amostra) =====")
    print(df_amostra.head(20))